In [ ]:
# standard imports
import numpy as np
import math
#import os
#import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh
from isentropic_navier_stokes import Q,L

# utility functions
from utilities_modified import *

# Setting Custom Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,[0.5,0])

# Data Preparation

In [ ]:
# reading file data
data_dir  = "./lam_cyl_comp_air/mach 0.1/"
data_dict = read_csv_data(data_dir,delay=0,subsampling=3,max_size=180)

In [ ]:
# geometric information
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]

# state based fluid properties
u = np.divide(data_dict['Momentum_x'],data_dict['Density'])
v = np.divide(data_dict['Momentum_y'],data_dict['Density'])
a = data_dict['Mach']

In [ ]:
# construct state based data
q = np.vstack([u,v,a])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

# Geometric Constants

In [ ]:
# DEFINING CONSTANTS
n = mesh.n      # number of nodes
N = mesh.N      # number of cells
T = q.shape[1]  # number of snapshots
alpha = 1       # inner product coefficient
method = "fd"     # method for derivative computation
gamma = 1.4     # isentropic coefficient -> material property
max_POD = 40

In [ ]:
skalar_weights = np.empty(n)
volume_weight = np.zeros(n)
for nod in mesh.nodes:
    volume_weight[nod.index] = nod.dv
mach_weight = (2 * alpha / (gamma - 1))
inner_weight = np.concatenate([volume_weight,volume_weight,mach_weight*volume_weight])

def skalar_product(d1,d2,w):
    return np.sum(d1*d2*w)

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

# POD Decomposition

In [ ]:
# computing correlation matrix with isenctropic inner product
C = np.empty((T,T))
for i in tqdm(range(T)):
    for j in range(i,T):
        C[i,j] = (1/T) * skalar_product(q_fluc[:,i],q_fluc[:,j],inner_weight)
        C[j,i] = C[i,j]  # C is symmetric by construction

In [ ]:
S, V =  np.linalg.eigh(C,UPLO='L')
S = np.flip(S,0)
V = np.flip(V,1)

# construct spatial POD Modes - not normalized
pod_modes = np.zeros((3*n,max_POD))
for i in range(max_POD):
    pod_modes[:,i] = np.matmul(q_fluc,V[:,i]) 

# normalize POD
for i in range(max_POD):
    pod_modes[:,i] /= np.sqrt(S[i]*T)

# computing eigenvalues
S = np.zeros(max_POD)
for i in range(max_POD):
    for j in range(T):
        S[i] += skalar_product(q_fluc[:,j],pod_modes[:,i],inner_weight)**2
    S[i] /= T

# energy reconstruction threshold
epsilon = 0.9
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1

# manual value
num = 4

In [ ]:
plot_cylinder_data(x,y,pod_modes[0*n:n,0],zoom=True)

In [ ]:
# incorrect because POD modes are not orthonormal in euclidean space but in problem space
coeffs = np.matmul(np.linalg.pinv(pod_modes),q_fluc)

In [ ]:
plot_activations(coeffs,4)

# Galerkin system

In [ ]:
# Galerkin system coefficients
# Computation of Galerkin system parameters
Qavg = Q(mesh,q_avg,q_avg,method=method)
Lavg = L(mesh,q_avg,method=method)

# initialie arrays for parameters
b1 = np.empty(num)
b2 = np.empty(num)
L1 = np.empty((num,num))
L2 = np.empty((num,num))
Q_ = [np.empty((num,num,)) for x in range(num)]

# compute L and Q operators for projection
Q_tmp1 = np.empty((num,3*n))
Q_tmp2 = np.empty((num,3*n))
Q_tmp3 = np.empty((num,num,3*n))
L_tmp = np.empty((num,3*n))

for i in tqdm(range(num)):
    Q_tmp1[i] = Q(mesh,q_avg,pod_modes[:,i],method=method)
    Q_tmp2[i] = Q(mesh,pod_modes[:,i],q_avg,method=method)
    L_tmp[i] = L(mesh,pod_modes[:,i],method=method)
    for j in range(num):
        Q_tmp3[i,j] = Q(mesh,pod_modes[:,i],pod_modes[:,j],method=method)

# compute ODE coefficients
for k in tqdm(range(num)):
    b1[k] = skalar_product(Lavg,pod_modes[:,k],inner_weight)
    b2[k] = skalar_product(Qavg,pod_modes[:,k],inner_weight)
    for i in range(num):
        L1[k,i] = skalar_product(L_tmp[i],pod_modes[:,k],inner_weight)
        L2[k,i] = skalar_product(np.add(Q_tmp1[i],Q_tmp2[i]),pod_modes[:,k],inner_weight)
        for j in range(num):
            Q_[k][i,j] = skalar_product(Q_tmp3[i,j],pod_modes[:,k],inner_weight)

In [ ]:
# initial conditions
a0 = coeffs[:num,20]

# sampling span for solver
t0 = 0
dt = 0.001
tmax = 1
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))

# static viscosity
nu =   1/100 #1.516e-5

# defining ode function for node based coefficients
def galerkin_system(t,a):
    # set global variables references
    global Q_, L1, L2, b1, b2, nu
    
    a_dot = np.empty_like(a)
    
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1)))
    return a_dot


In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol.message

In [ ]:
# Galerkin model based on discrete reconstruction as q = \bar{q}+\sum_i^n a_i(t)*\phi_i(x)
d = int(num/2)
fig,ax = plt.subplots(d,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span

# add plots over full time domain
for i in range(num):
    if num <= 2:
        ax[int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t,sol.y[i,:len(t)])
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))